In [1]:
import pandas as pd
%matplotlib inline
import datetime
import numpy as np
import pandas as pd

import nltk
from dsu.features import nlp
from dsu import plotter

/home/nandhini/.virtualenvs/analytics/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
irisDf = pd.read_csv('../data/tesseract_binary_extracted_text.csv', dtype={'Filename': str, 'text':str})

In [3]:
irisDf.head()

,fname,text
0,Test929.jpg,at aaigeee\nee Niasan a La4ul\n\nyou Stal ae\n...
1,Test835.jpg,YT LIBS\n\nGAY\nWe
2,Test2977.jpg,NaN
3,Test1012.jpg,es\nte)\naod\nRid
4,Test3798.jpg,—a_p


In [4]:
irisDf.text

0      at aaigeee\nee Niasan a La4ul\n\nyou Stal ae\n...
1                                     YT LIBS\n\nGAY\nWe
2                                                    NaN
3                                      es\nte)\naod\nRid
4                                                   —a_p
                             ...                        
234                                                  NaN
235                        1 a\nea\n\nOO\n\ncea\nRee eal
236                                             HUFFPOST
237    eae ee eis\nea RT laa icey eee\nyour religion ...
238                                                  NaN
Name: text, Length: 239, dtype: object

In [5]:
iristDf = irisDf.fillna("dummy", inplace=True)

In [6]:
#words = [ea.lower()  for each in irisDf.text for ea in each.split()]
#plotter.show_image(nlp.word_cloud(words))

In [7]:
from nltk.corpus import sentiwordnet as swn
synset_names = list()
for each in swn.all_senti_synsets():
    synset_names.append(each.synset._name.split('.')[0])


In [8]:
# Create features.. simple database lookup fails after cleaning the text.. 
irisDf['word_len'] = irisDf.text.apply(lambda x: len(x.split()))
irisDf['char_len'] = irisDf.text.apply(lambda x: len(x))

In [9]:
irisDf.text = irisDf.text.apply(lambda x: nltk.wordpunct_tokenize(x.replace('_', ' ').replace('-', ' ')))

irisDf.text = irisDf.text.apply(lambda x:nlp.filter_non_words(nlp.filter_stop_words(x)))

In [10]:
irisDf.text

0           Positive b harmony say lo el Inspiring de ae
1                                                 GAY We
2                                                  dummy
3                                              te es Rid
4                                                      p
                             ...                        
234                                                dummy
235                                               ea Ree
236                                                     
237    angry religion someone diet Gay ea eating Bein...
238                                                dummy
Name: text, Length: 239, dtype: object

In [11]:
from nltk.stem.snowball import SnowballStemmer
sstemmer = SnowballStemmer('english')
#print(sstemmer.stem('positive'))
import Levenshtein

#distances = dict(map(lambda x: (x, Levenshtein.distance(x, 'inspiring')), synset_names))
#closest_words = {k: v for k, v in sorted(distances.items(), key=lambda item: item[1]) }
#distances
#closest_words
singles = nlp.spell_correction(['harmony', 'say', 'ace', 'inspiring', 'b', 'go', 'elk', 'd', 'positive'], 
                               method=None,source_words=synset_names, measure='levenshtein')
singles = ['harmony', 'say', 'ace', 'inspiring', 'b', 'go', 'elk', 'd', 'positive']
singles = nlp.filter_non_words(singles)
nlp.sentiment(singles, 'sentiwordnet')

(1.25, 0.5)

In [12]:
import nltk

#irisDf.text  = irisDf.text.apply(lambda x: [find_corrections(ea) for ea in x.split()])

def find_sentiment(text, db='sentiwordnet'):
    words = ''.join(text).lower()
    #print(words)
    tokens=nltk.word_tokenize(words) #for tokenization, row is line of a file in which tweets are saved.
    if tokens == ['dummy']:
        return 0.0, 0.0
    singles = list()
    for plural in tokens:
        
        corrects = nlp.spell_correction([plural], method=None, source_words=synset_names, 
                                    measure='levenshtein')
        #print(plural, corrects)
        if corrects:
            singles.append(list(corrects[plural].items())[0][0])
        else:
            print(plural)
        
    print(words, singles)
    singles = nlp.filter_non_words(singles)
    pscore, nscore = nlp.sentiment(singles, db)
    print(pscore, nscore)
    return pscore, nscore

irisDf['sentiwordnet_scores'] = irisDf.text.apply(find_sentiment)


positive b harmony say lo el inspiring de ae ['positive', 'b', 'harmony', 'say', 'go', 'elk', 'inspiring', 'd', 'ace']
1.25 0.5
gay we ['gay', 'wee']
0.75 0.0
te es rid ['ten', 'os', 'rid']
0.5 0.25
p ['p']
0.0 0.0
age equality ['age', 'equality']
0.75 0.0
life play win game ['life', 'play', 'win', 'game']
0.75 0.0
go ya a ['go', 'mya', 'a']
0.0 0.0
closet they gay ca gay seer nothing being men ur of well course dress ['closet', 'then', 'gay', 'uca', 'gay', 'seer', 'nothing', 'being', 'mean', 'ur', 'off', 'well', 'course', 'dress']
3.5 0.75
closet month tribe welcome shame best er spend forever waiting missing home brown fear don another deserve you if year ['closet', 'month', 'tribe', 'welcome', 'shame', 'best', 'ern', 'spend', 'forever', 'waiting', 'missing', 'home', 'brown', 'fear', 'don', 'another', 'deserve', 'sou', 'in', 'year']
6.375 1.75
nothing feel way learned hard instead either you let say ['nothing', 'feel', 'way', 'earned', 'hard', 'unsteady', 'either', 'sou', 'let', 'say

cerise es cee ['cerise', 'os', 'wee']
1.5 0.0
chosen to if have ae be happy could ['chosen', 'go', 'in', 'have', 'ace', 'be', 'happy', 'cold']
1.625 1.0
keep al love and want you who calm ['keep', 'all', 'love', 'any', 'want', 'sou', 'rho', 'calm']
1.125 0.25
it te ae ad one ['in', 'ten', 'ace', 'ad', 'one']
1.25 0.25
s which do curious gay do colored which mal say thing sa gay realize days negative say smart pathetic annoying word only connotation homosexual used it mean that the to happy the way choose te what f brightly a stupid ludicrous you ['s', 'whish', 'do', 'curious', 'gay', 'do', 'colored', 'whish', 'male', 'say', 'thing', 'sa', 'gay', 'realize', 'days', 'negative', 'say', 'smart', 'pathetic', 'annoying', 'word', 'only', 'connotation', 'homosexual', 'used', 'in', 'mean', 'thai', 'then', 'go', 'happy', 'then', 'way', 'choose', 'ten', 'hat', 'f', 'rightly', 'a', 'stupid', 'judicious', 'sou']
6.25 5.25
female abandon ['female', 'abandon']
0.875 0.25
legal in we fifty ['legal', '

In [26]:
irisDf['afinn_scores'] = irisDf.text.apply(lambda x: nlp.sentiment(x.split(), db='afinn'))

In [25]:
from afinn import Afinn
af = Afinn()
nlp.sentiment('angry religion someone diet gay ea eating being like', db='afinn')

0.0

In [39]:
def predict_sentiment_only_sentiwordnet(row):
    print(row)
    pscore, nscore = row[4]
    afinn_score = row[5]
    
    if pscore == nscore:
        return 'Random'
    elif pscore > nscore:
        return 'Positive'
    else:
        return 'Negative'
#irisDf['Category'] = irisDf.apply(predict_sentiment, axis=1)




Filename                                                Test929.jpg
text                   Positive b harmony say lo el Inspiring de ae
word_len                                                         26
char_len                                                        137
sentiwordnet_scores                                     (1.25, 0.5)
afinn_scores                                                      7
Category                                                   Positive
Name: 0, dtype: object


TypeError: sum expected at most 2 arguments, got 3

In [40]:
def predict_sentiment_sum_scores(row):
    print(row)
    pscore, nscore = row[4]
    afinn_score = row[5]
    score = pscore +  nscore + afinn_score
    if score == 0:
        return 'Random'
    elif score > 0:
        return 'Positive'
    else:
        return 'Negative'
irisDf['Category'] = irisDf.apply(predict_sentiment_sum_scores, axis=1)

Filename                                                Test929.jpg
text                   Positive b harmony say lo el Inspiring de ae
word_len                                                         26
char_len                                                        137
sentiwordnet_scores                                     (1.25, 0.5)
afinn_scores                                                      7
Category                                                   Positive
Name: 0, dtype: object
Filename               Test835.jpg
text                        GAY We
word_len                         4
char_len                        15
sentiwordnet_scores    (0.75, 0.0)
afinn_scores                     0
Category                  Positive
Name: 1, dtype: object
Filename               Test2977.jpg
text                          dummy
word_len                          1
char_len                          5
sentiwordnet_scores      (0.0, 0.0)
afinn_scores                      0
Category         

In [44]:
def predict_sentiment_tree_model_scores(row):
    print(row)
    pscore, nscore = row[4]
    afinn_score = row[5]
    word_len = row[2]
    char_len = row[3]
    if word_len > 11 or char_len > 63:
        score = pscore + nscore
    else:
        score = pscore +  nscore + afinn_score
    if score == 0:
        return 'Random'
    elif score > 0:
        return 'Positive'
    else:
        return 'Negative'
irisDf['Category'] = irisDf.apply(predict_sentiment_sum_scores, axis=1)

Filename                                                Test929.jpg
text                   Positive b harmony say lo el Inspiring de ae
word_len                                                         26
char_len                                                        137
sentiwordnet_scores                                     (1.25, 0.5)
afinn_scores                                                      7
Category                                                   Positive
Name: 0, dtype: object
Filename               Test835.jpg
text                        GAY We
word_len                         4
char_len                        15
sentiwordnet_scores    (0.75, 0.0)
afinn_scores                     0
Category                  Positive
Name: 1, dtype: object
Filename               Test2977.jpg
text                          dummy
word_len                          1
char_len                          5
sentiwordnet_scores      (0.0, 0.0)
afinn_scores                      0
Category         

In [41]:
irisDf.describe()

,word_len,char_len,afinn_scores
count,239.000000,239.000000,239.000000
mean,11.702929,62.598326,0.267782
std,23.479824,131.988818,1.952010
min,1.000000,2.000000,-10.000000
25%,1.000000,5.000000,0.000000
50%,2.000000,6.000000,0.000000
75%,15.500000,79.500000,0.000000
max,291.000000,1611.000000,7.000000


In [45]:
irisDf = irisDf.rename(columns={'fname': 'Filename'})
irisDf.head()
irisDf[['Filename', 'Category']].to_csv('submission7_tree_model.csv', index=False)

In [46]:
irisDf[irisDf.Category=='Random']

,Filename,text,word_len,char_len,sentiwordnet_scores,afinn_scores,Category
2,Test2977.jpg,dummy,1,5,"(0.0, 0.0)",0.0,Random
4,Test3798.jpg,p,1,4,"(0.0, 0.0)",0.0,Random
7,Test3294.jpg,dummy,1,5,"(0.0, 0.0)",0.0,Random
8,Test100.jpg,dummy,1,5,"(0.0, 0.0)",0.0,Random
9,Test396.jpg,dummy,1,5,"(0.0, 0.0)",0.0,Random
...,...,...,...,...,...,...,...
232,Test212.jpg,dummy,1,5,"(0.0, 0.0)",0.0,Random
233,Test164.jpg,dummy,1,5,"(0.0, 0.0)",0.0,Random
234,Test3159.jpg,dummy,1,5,"(0.0, 0.0)",0.0,Random
236,Test238.jpg,,1,8,"(0.0, 0.0)",0.0,Random
